In [1]:
import pyaudio
import numpy as np
from queue import Queue
import threading
import atexit
import sounddevice as sd
import noisereduce as nr


In [2]:
# Constants for audio stream configuration
FORMAT = pyaudio.paInt16  # Sample format (16-bit)
CHANNELS = 1             # Number of audio channels (1 for mono, 2 for stereo)
# RATE = 44100             # Sample rate (samples per second)
CHUNK_SIZE = 44100        # Number of audio frames per buffer
RATE = 16000             # Sample rate (samples per second)

DETECT_THRESHOLD = 800   # Threshold to detect a clap
DETECT_DURATION = 0.5    # Time in seconds to detect a clap
RECORD_DURATION = 10      # Time in seconds to record audio after clap is detected

In [3]:
audio_input_queue = Queue(maxsize=2)
stop_record_event = threading.Event()


In [4]:
audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK_SIZE)

while True:
    data = stream.read(int(RECORD_DURATION * RATE/1000))
    # print(data[:20])
    audio_data = np.frombuffer(data, dtype=np.int16)

    print(audio_data[:20])
    print("detecting:", np.max(audio_data))

    if np.max(audio_data) < DETECT_THRESHOLD:
        continue

    audio_data_detect = audio_data

    data = stream.read(int(RECORD_DURATION * RATE))
    # print(data[:20])
    audio_data = np.frombuffer(data, dtype=np.int16)
    
    audio_data = np.concatenate((audio_data_detect, audio_data))

    
    # Reduce noise from the audio using noisereduce
    audio_data = nr.reduce_noise(y=audio_data, sr=RATE)

    break

[ 0  0 -1  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0]
detecting: 1
[-1  0  0  0  0  0 -1  0  0  0  0  0  0 -1  0  0  0  0  0  0]
detecting: 1
[ 0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0 -1  0  0 -1]
detecting: 1
[ 0  0  1  0  0  0  1 -1 -1  1  0  0  0 -1  0  0  1  0  0  0]
detecting: 1
[ 0  0  0 -1  1  1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0]
detecting: 1
[0 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0]
detecting: 1
[0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0]
detecting: 1
[ 0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0]
detecting: 1
[ 1 -1  0  0  1  1  0  0  1  0  0  1  0  0  1  0  1  0  0  0]
detecting: 2
[-2 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
detecting: 1
[-1  0  0 -1 -1  0 -1  0 -1  0 -1 -1  0 -1  0  0  1 -1  0  1]
detecting: 1
[-2 -2 -2 -2 -1  0 -1 -1 -1 -1 -2 -1 -2 -2 -2 -1 -1 -3 -2 -3]
detecting: 2
[-1  0 -1 -1  0  1 -1 -1 -1  1  1  1  0 -2  0 -2  1 -1 -1 -1]
detecting: 2
[-2 -4 -4 -3 -4 -4 -2 -3 -4 -2 -4 -4 -3 -4 -3 -3 -2 -3 -1 -2]
det

In [50]:
print(len(audio_data))

sd.play(audio_data, RATE)
sd.wait()

160160


In [1]:
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch

model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: c:\Users\benve\anaconda3\envs\py39\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary c:\Users\benve\anaconda3\envs\py39\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


In [56]:
# processor.tokenizer.set_target_lang("cmn-script_simplified")
# model.load_adapter("cmn-script_simplified")

processor.tokenizer.set_target_lang("jpn")
model.load_adapter("jpn")


# processor.tokenizer.set_target_lang("eng")
# model.load_adapter("eng")

In [57]:
# supported_lang = processor.tokenizer.vocab.keys()

# import re

# # List of words
# word_list = supported_lang

# # Word you want to search for
# search_word = "jpn"

# # Define the regular expression pattern
# pattern = r".*" + re.escape(search_word) + r".*"

# # Loop through the list of words and search for the word
# for word in word_list:
#     if re.search(pattern, word):
#         print(f"'{search_word}' found in '{word}'")

In [59]:
print(audio_data)

[  2   6   3 ... 127 108  91]


In [60]:
inputs = processor([audio_data*1.0], sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
# 'joe keton disapproved of films and buster also had reservations about the media'


In [61]:
print(transcription)

flv台の天kです


In [67]:
import torch
from transformers import pipeline

# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base", device=device,  return_timestamps="word"
    # "automatic-speech-recognition", model="openai/whisper-medium-jp", device=device,  return_timestamps="word"
)
pipe(
    audio_data,
    max_new_tokens=256,
    generate_kwargs={"task": "transcribe"},
    chunk_length_s=30,
    batch_size=8,
)

{'text': ' You are the only one.',
 'chunks': [{'text': ' You', 'timestamp': (29.98, 29.98)},
  {'text': ' are', 'timestamp': (29.98, 29.98)},
  {'text': ' the', 'timestamp': (29.98, 29.98)},
  {'text': ' only', 'timestamp': (29.98, 29.98)},
  {'text': ' one.', 'timestamp': (29.98, 29.98)}]}

In [2]:
# import os
# os.system("pip install git+https://github.com/openai/whisper.git")
# import gradio as gr
import whisper

# from share_btn import community_icon_html, loading_icon_html, share_js

# model = whisper.load_model("small")
model = whisper.load_model("medium")


In [6]:
def inference(audio):
    if type(audio) == str:
        audio = whisper.load_audio(audio)
    print(type(audio), audio.dtype, audio.shape)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    # _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    
    print(result.text)
    return result.text


In [7]:
print(audio_data)
# print(audio_data.astype(np.float32))
inference(audio_data.astype(np.float32))

[17 11 17 ...  4  2  5]
<class 'numpy.ndarray'> float32 (160160,)
今日の天気はいいです。何をしたのか?はぁ、悪い。


'今日の天気はいいです。何をしたのか?はぁ、悪い。'

In [127]:
inference("voice.wav")


<class 'numpy.ndarray'> float32 (244736,)
東京電極福島第一原発の処理水の環陽放出が24日にも始まる 放出に強く反発してきた中国からはすぐさま日本政府を批判する声があった


'東京電極福島第一原発の処理水の環陽放出が24日にも始まる 放出に強く反発してきた中国からはすぐさま日本政府を批判する声があった'